### RiskCalc examples.
Make sure that the working directory for this ipynb notebook is ./risktables/risktables

___
The class ```RiskCalcs``` performs basic portfolio risk calculations.  It returns a dictionary, whose items are either instances of Pandas ```DataFrame``` or floating point values representing portfolio aggregate risk measurements (VaR, Greeks, etc).
___

In [10]:
# include both the project package, and the project folder in sys.path
import sys,os
if  not './' in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not '../' in sys.path:
    sys.path.append(os.path.abspath('../'))

from risktables import risk_tables
import pandas as pd
from IPython import display
import datetime

#### Create an instance of RiskCalcs

In [9]:
df_postgres_info = pd.read_csv('postgres_info.csv')
display.display(df_postgres_info)
prices_db_info = df_postgres_info.loc[df_postgres_info.config_name=='dashrisk_local']
rt = risk_tables.RiskCalcs(use_postgres=False)

,config_name,dburl,databasename,username,password,schema_name,table_names
0,secdb_local,localhost,sec_db,NaN,NaN,sec_schema,options_table; underlying_table
1,dashrisk_local,localhost,maindb,NaN,NaN,test_schema,yahoo_daily


#### Create an example portfolio consisting of hypothetical options on the S&P 500 Sector Spdr's

In [20]:
next_year = (datetime.datetime.now() + datetime.timedelta(weeks=52)).year
next_year = str(next_year)
df_spdr_options =  pd.read_csv('spdr_stocks.csv')
df_spdr_options.symbol = df_spdr_options.symbol.apply(lambda s: s.replace('2021',next_year))
display.display(df_spdr_options)


,symbol,position
0,XLB_20201222_58_c,172
1,XLC_20201222_49_p,204
2,XLE_20201222_67_c,149
3,XLF_20201222_27_p,370
4,XLI_20201222_78_c,128
5,XLK_20201222_78_p,128
6,XLP_20201222_57_c,175
7,XLU_20201222_57_p,175
8,XLV_20201222_86_c,116
9,XLY_20201222_119_p,84


2020


#### Run risk calculations

In [21]:
risk_dictionary = rt.calculate(df_spdr_options)

Start computing VaR 2019-07-23 09:16:24.086343
entering hedge ratio calc
leaving hedge ratio calc
entering update_delta_div
End computing VaR 2019-07-23 09:16:54.105270
leaving update_memory


#### List the names of the outputs

In [23]:
for k in risk_dictionary.keys():
    print(k)

yyyymmddhhmmssmmmmmm
df_std
df_atm_price
df_high_low
df_atm_info
df_risk_all
df_risk_by_underlying
df_var
port_var
sp_dollar_equiv
delta
gamma
vega
theta
df_hedge_ratios
df_corr
df_corr_price


#### Now show the actual values of each key

In [29]:
for k in risk_dictionary.keys():
    print(f'********************* {k} **************************')
    dict_item = risk_dictionary[k]
    object_to_print = dict_item
    if type(dict_item)==dict:
        if k[:2] == 'df':
            object_to_print = risk_tables.make_df(risk_dictionary[k])
    display.display(object_to_print)


********************* yyyymmddhhmmssmmmmmm **************************


'20190723091654105270'

********************* df_std **************************


,stdev,underlying
0,0.195168,XLK
1,0.145961,XLF
2,0.176185,XLC
3,0.118887,XLV
4,0.177181,XLE
5,0.110703,XLP
6,0.163823,XLB
7,0.158597,XLI
8,0.133940,XLU
9,0.142247,XLY


********************* df_atm_price **************************


...

,hedge,symbol
0,240.780115,XLB
1,26.581594,XLP


********************* df_corr **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.356582,0.587470,0.783155,0.814978,0.610137,0.615200,0.098159,0.564137,0.654819
XLC,XLC,0.356582,1.000000,0.437241,0.528552,0.564428,0.814611,0.349750,-0.076628,0.565395,0.814654
XLE,XLE,0.587470,0.437241,1.000000,0.634038,0.648405,0.586792,0.247311,-0.227085,0.330246,0.591311
XLF,XLF,0.783155,0.528552,0.634038,1.000000,0.822377,0.726773,0.481125,0.019851,0.631213,0.790314
XLI,XLI,0.814978,0.564428,0.648405,0.822377,1.000000,0.750529,0.467739,-0.059325,0.568712,0.807501
XLK,XLK,0.610137,0.814611,0.586792,0.726773,0.750529,1.000000,0.439863,-0.120407,0.653923,0.898315
XLP,XLP,0.615200,0.349750,0.247311,0.481125,0.467739,0.439863,1.000000,0.520531,0.679174,0.522794
XLU,XLU,0.098159,-0.076628,-0.227085,0.019851,-0.059325,-0.120407,0.520531,1.000000,0.363948,-0.088600
XLV,XLV,0.564137,0.565395,0.330246,0.631213,0.568712,0.653923,0.679174,0.363948,1.000000,0.658112
XLY,XLY,0.654819,0.814654,0.591311,0.790314,0.807501,0.898315,0.522794,-0.088600,0.658112,1.000000


********************* df_corr_price **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.563157,0.356733,0.790069,0.845421,0.856218,0.890452,0.807478,0.926951,0.830672
XLC,XLC,0.563157,1.000000,0.785438,0.855531,0.824851,0.874197,0.714666,0.491360,0.670678,0.899400
XLE,XLE,0.356733,0.785438,1.000000,0.660677,0.733270,0.656923,0.446224,0.188233,0.429747,0.672658
XLF,XLF,0.790069,0.855531,0.660677,1.000000,0.913837,0.939570,0.827435,0.608665,0.785267,0.938948
XLI,XLI,0.845421,0.824851,0.733270,0.913837,1.000000,0.944824,0.813553,0.605222,0.835804,0.935991
XLK,XLK,0.856218,0.874197,0.656923,0.939570,0.944824,1.000000,0.897547,0.684793,0.860802,0.982863
XLP,XLP,0.890452,0.714666,0.446224,0.827435,0.813553,0.897547,1.000000,0.886613,0.884414,0.896747
XLU,XLU,0.807478,0.491360,0.188233,0.608665,0.605222,0.684793,0.886613,1.000000,0.861219,0.682617
XLV,XLV,0.926951,0.670678,0.429747,0.785267,0.835804,0.860802,0.884414,0.861219,1.000000,0.842099
XLY,XLY,0.830672,0.899400,0.672658,0.938948,0.935991,0.982863,0.896747,0.682617,0.842099,1.000000
